In [1]:
import datetime
from pycti import OpenCTIApiClient
from dateutil.parser import parse
import pandas as pd
import os
import warnings

In [5]:
api_url = "https://demo.opencti.io"
api_token = "609caced-7610-4c84-80b4-f3a380d1939b"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)


INFO:root:Listing Threat-Actors with filters null.


In [73]:
class entity_insertion:
    
   
        

    def __init__(self,report_path,stix_domain= dict()):
    
        self.report_path = report_path
        self.stix_domain = stix_domain
        
        
        
    
    def add_mark_def(self,row,stix_id):
        mark      =row['marking_definitions']
        all_marks =opencti_api_client.marking_definition.list()
        for i in all_marks:
            if mark in i['definition'].lower():
                mark_code= i['id']
                opencti_api_client.stix_entity.add_marking_definition(id=stix_id,marking_definition_id=mark_code)  
     
    def create_threat_actor(self,by,report):
            
            threat_actor_details = pd.read_excel(self.report_path,sheet_name = 'threat_actor')
            threat_actor_details = threat_actor_details.where(threat_actor_details.notnull(), None)
            
            for index,row in threat_actor_details.iterrows():
            
                
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    
                
                threat_actor = opencti_api_client.threat_actor.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                goal = row['goal'] ,
                                                sophistication = row['sophistication'] ,
                                                resource_level = row['resource_level'] ,
                                                primary_motivation = row['primary_motivation'] ,
                                                secondary_motivation = row['secondary_motivation'] ,
                                                personal_motivation = row['personal_motivation'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                update = False
                                                 )
                    
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = threat_actor['id'] )       
                self.stix_domain.update ({row['name'].strip() : threat_actor['id']})
                        
    def create_attack_pattern(self,by,report):           #attack_pattern insertion    
            
            attack_pattern_details = pd.read_excel(self.report_path,sheet_name = 'attack_pattern')
            attack_pattern_details = attack_pattern_details.where(attack_pattern_details.notnull(), None)
            
            for index,row in attack_pattern_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    
                if row['platform'] is not None:
                    platforms =row['platform'].split(',')
                else:
                    platforms = None
                    
                print(row['required_permission'])   
                
                exist = opencti_api_client.attack_pattern.list ( filters=[{'key' : 'name' , 'values' : row['name']}])

                
                if (len(exist) == 0):
                    attack_pattern = opencti_api_client.attack_pattern.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases,
                                                platform  = platforms ,
                                                required_permission = row['required_permission'] ,
                                                external_id  = row['external_id'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     attack_pattern = opencti_api_client.attack_pattern.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                platform  = platforms ,
                                                required_permission = row['required_permission'] ,
                                                external_id  = row['external_id'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=attack_pattern['id'],row=row)
                        
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = attack_pattern['id'] )  
                self.stix_domain.update ({row['name'].strip() : attack_pattern['id']})      
                        
    def create_malware(self,by,report):           #malware insertion    
            
            malware_details = pd.read_excel(self.report_path,sheet_name = 'malware')
            malware_details = malware_details.where(malware_details.notnull(), None)
            
            for index,row in malware_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.malware.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=malware['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = malware['id'] )
                self.stix_domain.update ({row['name'].strip() : malware['id']}) 
                
    def create_tool(self,by,report):           #malware insertion    
            
            tool_details = pd.read_excel(self.report_path,sheet_name = 'tool')
            tool_details = tool_details.where(tool_details.notnull(), None)
            
            for index,row in tool_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.tool.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=tool['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = tool['id'] )
                self.stix_domain.update ({row['name'].strip() : tool['id']})
                
    def create_vulnerabilty(self,by,report):           #malware insertion    
            
            vul_details = pd.read_excel(self.report_path,sheet_name = 'vulnerability')
            vul_details = vul_details.where(vul_details.notnull(), None)
            
            for index,row in vul_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.vulnerability.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=vulnerability['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = vulnerability['id'] )
                self.stix_domain.update ({row['name'].strip() : vulnerability['id']})  
                
    def create_indicator(self,by,report):           #malware insertion    
            
            observable_details = pd.read_excel(self.report_path,sheet_name = 'observable')
            observable_details = vul_details.where(observable_details.notnull(), None)
            
            for index,row in observable_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.vulnerability.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=vulnerability['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = vulnerability['id'] )
                self.stix_domain.update ({row['name'].strip() : vulnerability['id']})  
                
    def create_report(self):

        report_details = pd.read_excel(self.report_path,sheet_name = 'report')
        report_details = report_details.where(report_details.notnull(), None)
        
        # report creation
        for index,row in report_details.iterrows():
            report_by = opencti_api_client.identity.list(filters=[{'key':'name','values': row['created_by_ref'] }])
           
            if len(report_by) == 0 : 
                warnings.warn("Since " + row['created_by_ref'] + " is not inserted in opencti.Rhymtechnologies is taken as default")
                report_by = organization = opencti_api_client.identity.create(
                                            type='Organization',
                                            name="Rhymtechnologies",
                                            alias=["Rhymtech"],
                                            description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                            )
            else:
                report_by = report_by[0]
                
           # report_by = {'id': None}
                
            
            print(row ['source_confidence_level'])
            
            report = opencti_api_client.report.create(
                                                            name=row['name'],
                                                            description= row['description'],
                                                            report_class="Internal Report",
                                                            published=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                                                            createdByRef= report_by['id'],
                                                            object_status = row['object_status'],
                                                            source_confidence_level = row ['source_confidence_level'],
                                                            update = True,
                                                            )
            if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
            print("one_completed")
            
            
            self.create_threat_actor(by = report_by['id'] ,report = report['id'])  
            self.create_attack_pattern(by = report_by['id'] , report =report ['id'])
            self.create_malware (by = report_by['id'] , report = report ['id'])
            self.create_tool (by = report_by['id'] , report = report ['id'])
            self.create_vulnerabilty (by = report_by['id'] , report = report ['id'])
            
            return self.stix_domain
                 
                
            

In [74]:
en=entity_insertion(report_path=r"D:\Threat_intel_services\srikanth\fakeapt12.xlsx")

In [75]:
en.create_report()

INFO:root:Listing Identities with filters [{"key": "name", "values": "Rhymtech"}].
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:267: UserWarning: Since Rhymtech is not inserted in opencti.Rhymtechnologies is taken as default
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Rhymtechnologies"]}].
INFO:root:Listing Reports with filters [{"key": "name", "values": ["fakeapt12"]}, {"key": "published_day", "values": ["2020-01-29"]}].


1.0


INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["fakeapt12"]}].


one_completed


INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Attack-Patterns with filters [{"key": "name", "values": "Spear phishing"}].


...

['bbsinfo', 'aumlib']


INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Aumlib"]}].
INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Malwares with filters [{"key": "name", "values": "Threebyte"}].
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Threebyte"]}].
INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Malwares with filters [{"key": "name", "values": "Hightide"}].


['Etumbot']


INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Hightide"]}].
INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Malwares with filters [{"key": "name", "values": "Waterspout"}].
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Waterspout"]}].
INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Tools with filters [{"key": "name", "values": "HTRAN"}].
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["HTRAN"]}].
INFO:root:Reading Report {b694b5ff-4dee-4a88-8102-4ecefdbaf902}.
INFO:root:Listing Vulnerabilities with filters [{"key": "name", "values": "CVE-2009-4324"}].
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["CVE-2009-4324"]}].
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "alias", "values": ["CVE-2009-4324"]}].
INFO:root:Creating Tool {CVE-2009-4324}.
INFO:root:Reading Report {b6

{'fakeapt12': '4e459911-9c58-4add-b147-5c33e72d11e2',
 'Spear phishing': 'ae0eebe7-6e24-40e5-a5d2-59e2478086ca',
 'Exploitation for Client Execution': 'ed27d3c8-e054-49ff-80d1-19c0d72dc286',
 'Spearphishing attachment': '3a2e89b6-4dbd-4172-8340-b424dff022f9',
 'User Execution': 'aa4dac29-3d7a-449c-9455-28977b7f27a2',
 'Web Service': '3b5277c0-34c5-4371-8516-3aeee9c14d69',
 'Command-Line Interface': '7aa0b906-0670-4d97-a0f3-00fb626856d4',
 'Commonly Used Port': 'fab5312a-b85a-4b52-961c-a18deed6a0ef',
 'Data from Local System': '97d127c9-761b-422f-b2af-f162779e2c72',
 'Data Obfuscation': 'd1aa53d0-9808-4325-b46a-612b83212b4a',
 'File and Directory Discovery': '3b6464bf-890d-494a-835f-f5f8cc4b2d58',
 'File Deletion': '5d0c505b-999d-4039-8e18-a704b234f972',
 'Hidden Files and Directories': '7df7f45d-47f9-4e60-b984-a12cfcd4e034',
 'Masquerading': 'ba2deef8-4c2f-4346-b412-36a148697fec',
 'Process Discovery': '637c1675-6cff-479b-abf2-a6828ee06f49',
 'Registry Run Keys / Startup Folder': 'cdc0

In [25]:
stix_domain

NameError: name 'stix_domain' is not defined

In [ ]:
list(a)

In [ ]:
a=[]
if a is None:
    print(1)

In [ ]:
def create_tool(self,by,report):           #malware insertion    
            
            tool_details = pd.read_excel(self.report_path,sheet_name = 'tool')
            tool_details = tool_details.where(tool_details.notnull(), None)
            
            for index,row in tool_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.tool.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) > 0):
                    tool = opencti_api_client.tool.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     tool = opencti_api_client.tool.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=tool['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = tool['id'] )

In [ ]:
def create_report(self,report_by):

    report_details=pd.read_excel(self.report_path,sheet_name = 'report')
    report_by = opencti_api_client.identity.read(filters=[{'key':'name','values': report_by }])
    
    if report_by is None: 
        warnings.warn("Since " + report_by + " is not inserted in opencti.Rhymtechnologies is taken as default")
        report_by = organization = opencti_api_client.identity.create(
                                    type='Organization',
                                    name="Rhymtechnologies",
                                    alias=["Rhymtech"],
                                    description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                    )
                        
    
    
    for index,row in report_details.iterrows():
        
        date = parse(row['published']).strftime("%Y-%m-%dT%H:%M:%SZ")
        if source_confidence_level is None:
                    report = opencti_api_client.report.create(
                                                    name=row['name'],
                                                    description= row['description'],
                                                    report_class="Internal Report",
                                                    published=date,
                                                    createdByRef= report_by['id'],
                                                    object_status = 0,
                                                    update = True,
                                                    )
        else:
             report = opencti_api_client.report.create(
                                                    name=row['name'],
                                                    description= row['description'],
                                                    report_class="Internal Report",
                                                    published=date,
                                                    createdByRef= report_by['id'],
                                                    object_status = row['object_status'],
                                                    source_confidence_level = row ['source_confidence_level'],
                                                    update = True,
                                                    )


In [ ]:
create_report()